In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

# 基本用法

## 例子1：简单的invoke

In [50]:
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph
from langgraph.graph import END
import operator

# 定义状态
class AgentState(TypedDict):
    messages: Annotated[Sequence[str], operator.add]

# 定义图
workflow = StateGraph(AgentState)

# 定义节点
def nodeA(state):
    return {"messages": ["AAA"]}

workflow.add_node("a", nodeA)

# 定义节点
def nodeB(state):
    return {"messages": ["BBB"]}

workflow.add_node("b", nodeB)

# 定义边
workflow.add_edge("a", "b")
workflow.add_edge("b", END)
workflow.set_entry_point("a")

# 
app = workflow.compile()

# RUN
app.invoke({"messages": []})

{'messages': ['AAA', 'BBB']}

## 例子2：MessageGraph

In [203]:
from langchain.llms.fake import FakeStreamingListLLM
llm = FakeStreamingListLLM(responses=["你好，我是一个模拟的大语言模型"])

In [204]:
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, MessageGraph, END
import operator

# 定义图
workflow = MessageGraph()

# 定义节点
def nodeA(state):
    return ["AAA"]

workflow.add_node("a", nodeA)

# 定义节点
def nodeB(state):
    return ["BBB"]

workflow.add_node("b", llm)

# 定义边
workflow.add_edge("a", "b")
workflow.add_edge("b", END)
workflow.set_entry_point("a")

# 
app = workflow.compile()

In [205]:
for s in app.stream([]):
    print(s)

{'a': ['AAA']}
{'b': '你好，我是一个模拟的大语言模型'}
{'__end__': ['AAA', '你好，我是一个模拟的大语言模型']}


In [206]:
# RUN
async for chunk in app.astream_events([], version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="b"):
        print(">"*10, chunk['data']['chunk'])

>>>>>>>>>> 你
>>>>>>>>>> 好
>>>>>>>>>> ，
>>>>>>>>>> 我
>>>>>>>>>> 是
>>>>>>>>>> 一
>>>>>>>>>> 个
>>>>>>>>>> 模
>>>>>>>>>> 拟
>>>>>>>>>> 的
>>>>>>>>>> 大
>>>>>>>>>> 语
>>>>>>>>>> 言
>>>>>>>>>> 模
>>>>>>>>>> 型


## 例子3：Chain

In [189]:
from langchain.llms.fake import FakeStreamingListLLM
llm = FakeStreamingListLLM(responses=["你好，我是一个模拟的大语言模型"])

In [190]:
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="messages")
])

In [192]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    { "messages": lambda x: x['messages'] }
    | chat_prompt
    | llm
)

In [202]:
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, MessageGraph, END
import operator

# 定义状态
class AgentState(TypedDict):
    messages: Annotated[Sequence[str], operator.add]

# 定义图
workflow = StateGraph(AgentState)

# 定义节点
def nodeA(state):
    return {"messages": ["AAA"]}

workflow.add_node("__Node_A__", nodeA)

# 定义节点
async def nodeB(state):
    messages = state["messages"]
    response = await llm.ainvoke(messages)
    return {"messages": [response]}

workflow.add_node("__Node_B__", nodeB)

# 定义边
workflow.add_edge("__Node_A__", "__Node_B__")
workflow.add_edge("__Node_B__", END)
workflow.set_entry_point("__Node_A__")

# 
app = workflow.compile()

# RUN
# app.invoke({"messages": []})
async for chunk in app.astream_events({"messages": []}, version="v1"):
    print(" "*10, chunk['name'], chunk['event'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="__Node_B__"):        
        print(">"*10, chunk['data']['chunk'])

           LangGraph on_chain_start []
           __start__ on_chain_start ['graph:step:0', 'langsmith:hidden']
           __start__ on_chain_stream ['graph:step:0', 'langsmith:hidden']
           __start__ on_chain_end ['graph:step:0', 'langsmith:hidden']
           __start__ on_chain_start ['graph:step:0', 'langsmith:hidden']
           __start__ on_chain_end ['graph:step:0', 'langsmith:hidden']
           __start__:edges on_chain_start ['graph:step:1', 'langsmith:hidden']
           ChannelRead<['messages']> on_chain_start ['seq:step:1', 'langsmith:hidden']
           ChannelRead<['messages']> on_chain_stream ['seq:step:1', 'langsmith:hidden']
           ChannelWrite<__Node_A__:inbox> on_chain_start ['seq:step:2', 'langsmith:hidden']
           ChannelWrite<__Node_A__:inbox> on_chain_stream ['seq:step:2', 'langsmith:hidden']
           __start__:edges on_chain_stream ['graph:step:1', 'langsmith:hidden']
           ChannelRead<['messages']> on_chain_end ['seq:step:1', 'langsmith:hidd

# Function-Calling

## 准备

## 简单的例子

### 定义图

In [1]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [2]:
# 定义状态
workflow = StateGraph(AgentState)

### 定义工具集

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

### 定义 agent 节点

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)

In [8]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tools]
model = model.bind(functions=functions)

In [9]:
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [10]:
# 定义 agent 节点
workflow.add_node("agent", call_model)

### 定义 action 节点

In [11]:
from langchain_core.messages import FunctionMessage
from langgraph.prebuilt import ToolInvocation
import json

from langgraph.prebuilt import ToolExecutor
tool_executor = ToolExecutor(tools)

def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

In [12]:
# 定义 action 节点
workflow.add_node("action", call_tool)

### 定义条件边

In [13]:
from langgraph.graph import END

def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

# 增加边
workflow.add_conditional_edges(
    "agent", # from节点
    should_continue, # 条件
    {
        # to节点：回到 action
        "continue": "action",
        # to节点：结束
        "end": END
    }
)

### 定义直连边

In [14]:
workflow.add_edge('action', 'agent')

### 设定执行入口

In [15]:
workflow.set_entry_point("agent")

### 编译 graph

In [16]:
app = workflow.compile()

### 使用它！

In [18]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf?")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}}),
  FunctionMessage(content="[{'url': 'https://www.whereandwhen.net/when/north-america/california/san-francisco-ca/february/', 'content': 'Best time to go to San Francisco? Weather in San Francisco in february 2024  How was the weather last february? Here is the day by day recorded weather in San Francisco in february 2023:  Seasonal average climate and temperature of San Francisco in february  The climate of San Francisco in february is goodSan Francisco at the Beginning of February. Temperatures ranging from 9° C to 14° C translate into very mild mornings and pleasant days. However, chilly people\\xa0...'}]", name='tavily_search_results_json'),
  AIMessage(content="I'm sorry, but I couldn't find the current weather in San Francisco. However, in February, the wea

# langgraph示例

## 定义图

In [18]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

# 定义状态
workflow = StateGraph(AgentState)

## 定义开始节点

In [22]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)

def speak_joke(state):
    messages = state['messages']
    response = model.stream('讲一个30字的笑话')
    # We return a list, because this will get added to the existing list
    for chunk in response:
        yield {"messages": [chunk]}

workflow.add_node("speak_joke", speak_joke)

## 定义结束节点

In [23]:
def listen_joke(state):
    messages = state['messages']
    for m in messages:
        print(m)
    return {"messages": "hahaha"}

workflow.add_node("listen_joke", listen_joke)

## 定义边

In [24]:
from langgraph.graph import END

workflow.add_edge('speak_joke', 'listen_joke')
workflow.add_edge('listen_joke', END)

## 编译并使用

In [ ]:
workflow.set_entry_point("speak_joke")
app = workflow.compile()
for x in app.stream({}):
    print(x)

# chat_agent_executor

In [2]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import chat_agent_executor
from langchain_core.messages import HumanMessage

tools = [TavilySearchResults(max_results=1)]
model = ChatOpenAI(streaming=True)

app = chat_agent_executor.create_tool_calling_executor(model, tools)

In [3]:
inputs = {"messages": [HumanMessage(content="霍金的生日是哪一天？")]}

In [ ]:
for chunk in app.stream(inputs):
    print(chunk)

In [ ]:
async for output in app.astream_log(inputs, include_types=["llm"]):
    # astream_log() yields the requested logs (here LLMs) in JSONPatch format
    for op in output.ops:
        if op["path"] == "/streamed_output/-":
            # this is the output from .stream()
            ...
            print(op["value"])
        elif op["path"].startswith("/logs/") and op["path"].endswith(
            "/streamed_output/-"
        ):
            # because we chose to only include LLMs, these are LLM tokens
            print(op["value"])

In [ ]:
async for chunk in app.astream_events(inputs, version="v1"):
    print("\n")
    print("-"*80)
    print(f"name: {chunk['name']}")
    print(f"tags: {chunk['tags']}")
    print(f"event: {chunk['event']}")
    if(chunk['event']=="on_chain_stream"):
        m = chunk['data']['chunk']
        if "messages" in m:
            for x in m["messages"]:
                print(x.content, end="|", flush=True)


# 从智能体中提取对话信息

您可以通过定义一个“记忆器”（通常可以是一个队列或者其他类型的数据结构，用于临时存储数据）来实现这个功能。这种方法允许您在服务器端持续将数据写入到这个记忆器中，同时客户端可以异步地读取这些数据，直到所有流式结果都被处理完毕。这种模式在处理实时数据流、日志监控或者其他需要实时数据更新的场景中非常有用。

以下是实现这一功能的大致步骤和示例代码：

步骤 1：定义记忆器
记忆器可以是一个简单的队列（如 Python 的 queue.Queue），用于在生产者（数据生成函数）和消费者（客户端请求处理函数）之间传递消息。

步骤 2：数据生成函数循环写入记忆器
您的数据生成函数（或者其他逻辑）会在循环中生成数据，并将这些数据写入到记忆器中。

步骤 3：客户端异步读取记忆器
通过一个特定的 API 端点，客户端可以开始读取记忆器中的数据。服务器端可以使用异步处理来实时发送记忆器中的数据给客户端，直到所有数据都被发送。

示例代码
```python
from fastapi import FastAPI, WebSocket
import asyncio
from typing import List

app = FastAPI()

# 定义一个简单的记忆器，这里使用列表模拟
memory = []

# 数据生成函数，模拟数据产生并写入记忆器
async def generate_data():
    for i in range(10):
        await asyncio.sleep(1)  # 模拟异步操作
        memory.append(f"Data {i}")  # 将数据写入记忆器

# WebSocket路由，客户端通过WebSocket连接读取数据
@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    # 启动数据生成任务，但不等待它完成
    task = asyncio.create_task(generate_data())
    while not task.done() or memory:  # 检查任务是否完成或记忆器中是否有数据
        if memory:  # 如果记忆器中有数据，则发送给客户端
            data_to_send = memory.pop(0)  # 获取记忆器中的第一个数据并发送
            await websocket.send_text(data_to_send)
        else:
            await asyncio.sleep(0.1)  # 等待更多数据生成
    await websocket.close()
```
在这个示例中，generate_data 函数模拟了数据生成过程，将数据逐个写入全局变量 memory 中。通过 WebSocket 连接，客户端可以实时地接收这些数据。服务器在有数据可发送时通过 WebSocket 向客户端发送数据，直到数据生成任务完成并且记忆器中的所有数据都被发送。

请注意，这里使用全局变量作为记忆器仅用于示